In [6]:
search_for = 107
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016973495483398438
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 132316201
type: [1, 1, 1, 1, 107] 107
cases of this type: 131079601
10000 18.456760000580193
20000 19.740819830897035
30000 21.37634974735927
40000 65.77386797479058
50000 103.93726848900363
60000 90.14645914865915
70000 84.53403289800585
80000 93.90096334486506
90000 91.8159271948726
100000 88.29095668910193
110000 89.18829579052968
120000 95.17014088642898
130000 91.05873966074431
140000 87.53651159406256
150000 92.53068707750275
160000 91.75575660831566
170000 89.984581771265
180000 87.17360295877188
190000 88.8504364987455
200000 89.79730600624012
210000 85.65010619703854
220000 85.11555883468306
230000 88.91324487004003
240000 91.58766784901347
250000 85.8906612642697
260000 91.33546886747332
270000 87.519052033996
280000 89.1894362127524
290000 86.75032311366196
300000 97.23659142243858
310000 97.2973836136815
320000 101.41

3120000 92.75781998529345
3130000 86.82299657116822
3140000 78.98289529978466
3150000 77.93772040306453
3160000 86.02865036480193
3170000 84.68718293978652
3180000 78.5583434345287
3190000 79.07818303559239
3200000 84.56919766127699
3210000 82.85652491210612
3220000 78.6132751067502
3230000 79.82998745690635
3240000 83.31532536713217
3250000 79.1021203833316
3260000 77.17300432760139
3270000 78.88354603457057
3280000 78.79588302885412
3290000 78.1698993831658
3300000 81.48168138449067
3310000 82.37231354947119
3320000 83.7659941197612
3330000 79.406784535219
3340000 85.17026606644184
3350000 81.3554432126309
3360000 84.89605480352955
3370000 87.15476317444188
3380000 99.7096482072369
3390000 85.69584270185825
3400000 91.6303296747084
3410000 79.26993621122207
3420000 84.95924588489248
3430000 82.49000750911296
3440000 84.96904073449434
3450000 81.96923107194824
3460000 80.63301524112057
3470000 78.9701248617102
3480000 85.88485779125513
3490000 81.42463331310611
3500000 79.763274616616

6280000 84.2467906644546
6290000 86.55915872173678
6300000 79.6146075041486
6310000 80.60529332153148
6320000 80.60823711072214
6330000 77.82683536600908
6340000 76.72423284725868
6350000 77.49311749473773
6360000 81.6239345094727
6370000 81.67582843216447
6380000 80.79283150761235
6390000 85.1906031412834
6400000 86.93464634219295
6410000 89.80295574234707
6420000 87.78584224011554
6430000 83.76561378831836
6440000 82.16286356551355
6450000 84.74498142173303
6460000 90.28866945360163
6470000 83.34029754247523
6480000 93.10849936100054
6490000 86.68635232560659
6500000 79.63190077460273
6510000 81.15028878144084
6520000 80.97534495324719
6530000 77.04823318452853
6540000 79.84967659710067
6550000 77.7107599377802
6560000 79.15764007055526
6570000 77.94027913481533
6580000 80.18402737061143
6590000 82.36129518376629
6600000 84.17256239690913
6610000 84.37237252731886
6620000 80.76628685922512
6630000 84.51496554508265
6640000 85.7849644500283
6650000 86.46753315109386
6660000 76.1116989

9440000 79.03810648559616
9450000 86.43195712463668
9460000 80.62626712048417
9470000 79.45385416931151
9480000 81.01166589252571
9490000 78.31146708996614
9500000 79.08545984486535
9510000 82.18985047374552
9520000 78.15485133641309
9530000 81.18470734326957
9540000 80.3465239286216
9550000 79.113218989199
9560000 79.70604020651815
9570000 77.03519801581724
9580000 79.42836506126407
9590000 75.96666228486575
9600000 78.35210589890598
9610000 79.91749793137785
9620000 77.9292141279384
9630000 80.75300965560042
9640000 82.39746729779648
9650000 82.96269729230964
9660000 76.32832055609872
9670000 79.12726366203051
9680000 89.54152155432938
9690000 63.1193653354804
9700000 27.03222074596125
9710000 26.6030415670118
9720000 27.606059501962218
9730000 27.439712877569796
9740000 27.008293226023817
9750000 77.76011378173605
9760000 97.61895238159559
9770000 80.3398656118097
9780000 74.80791142217498
9790000 79.92042307050885
9800000 87.84534631172214
9810000 85.06230368870955
9820000 88.86088

12500000 85.95531748399029
12510000 80.38352807501622
12520000 81.0142223853383
12530000 79.57994197141107
12540000 78.2206998675712
12550000 80.09541983514961
12560000 78.728601988032
12570000 75.24443580089995
12580000 76.22189964619378
12590000 77.03117962780738
12600000 77.1375918909778
12610000 80.95872390592146
12620000 81.29285509169569
12630000 79.43121640329393
12640000 85.3955498086227
12650000 75.08790254999265
12660000 76.2669746552626
12670000 80.04252437504285
12680000 75.19762091755229
12690000 75.80805129185032
12700000 77.79817070426577
12710000 71.94152441885625
12720000 73.30576277251794
12730000 75.61299339250183
12740000 73.81159582328831
12750000 72.32260760996652
12760000 75.81019354235329
12770000 76.83124248461631
12780000 70.11573327229767
12790000 68.59344413773655
12800000 72.5115346250214
12810000 72.33437720552283
12820000 73.44427746832179
12830000 82.86073850990981
12840000 48.28410802626231
12850000 10.650075358149568
12860000 10.604633142634636
1287000

...

120030000 7.511631233456665
120040000 7.163008986486673
120050000 7.168691079578294
120060000 7.7410934492227375
120070000 8.239930546737135
120080000 7.536529517275175
120090000 6.639797288745219
120100000 7.1917975481722545
120110000 7.442079008203751
120120000 8.208348163033671
120130000 4.258237670582732
120140000 2.514004004801148
120150000 2.4091082721212635
120160000 2.189743320249855
120170000 2.3277642376323207
120180000 2.659694184258984
120190000 9.346273589732945
120200000 7.276300286556376
120210000 7.0357619893481065
120220000 7.1443726456593275
120230000 7.105340886991686
120240000 6.433349398810811
120250000 7.1458487737938166
120260000 6.6436062984666835
120270000 6.895584596575345
120280000 7.087976201176207
120290000 7.002225892326791
120300000 7.738482231727951
120310000 7.448574404187659
120320000 6.844860838777204
120330000 6.628314059717854
120340000 5.827097928660214
120350000 6.683705087036675
120360000 7.032545729714419
120370000 6.78767972744445
120380000 6.3

122950000 5.4950984157979486
122960000 4.892131383982016
122970000 5.731386343975067
122980000 5.8299465186839505
122990000 5.594454022437069
123000000 5.641039013581302
123010000 5.286724919920186
123020000 5.282976562115497
123030000 5.360069959672935
123040000 4.901408691556692
123050000 4.685077711827576
123060000 4.553688205472602
123070000 4.911063967420359
123080000 5.111204323913091
123090000 5.304403417100217
123100000 0.7887045180827579
123110000 0.7393640893276201
123120000 0.7309810408339104
123130000 0.7004375762965679
123140000 0.7013346984751025
123150000 3.424582411348747
123160000 6.021768606733004
123170000 5.227616684803048
123180000 4.347676159340117
123190000 5.248510452501813
123200000 5.1527406798290745
123210000 5.341252123051543
123220000 5.220965523903131
123230000 5.526984038719191
123240000 5.344180573421783
123250000 5.192853946581642
123260000 5.199265134422428
123270000 5.365427608247605
123280000 5.445814826963882
123290000 5.240595832527936
123300000 5.

125860000 3.728582890085022
125870000 4.093712234148477
125880000 3.7210305349067916
125890000 4.01916864761291
125900000 4.013419116011905
125910000 4.264174778393394
125920000 3.8090488426600095
125930000 3.804274817281564
125940000 3.8340916751994163
125950000 4.006242731430392
125960000 3.710398970575505
125970000 3.6233986698097724
125980000 3.7162375858377215
125990000 3.9993325770551293
126000000 3.472428443547011
126010000 3.398174567665637
126020000 3.6855468268332414
126030000 3.6902882823305467
126040000 3.6682306683839756
126050000 3.72007163265517
126060000 3.407580944293082
126070000 3.785598070005655
126080000 3.8296407043585843
126090000 3.8586291644526054
126100000 4.310825570381522
126110000 1.0267885204596388
126120000 1.008154140898009
126130000 0.9689530581708946
126140000 1.0163373942883942
126150000 1.0158652789156768
126160000 2.363591556978901
126170000 4.420676053652691
126180000 3.8257549259387984
126190000 4.555668387202382
126200000 1.3591809320029484
12621

128730000 2.196749891749349
128740000 2.215883124638796
128750000 2.2129829439582758
128760000 2.394286675116109
128770000 2.0882740775953335
128780000 2.252462300438391
128790000 2.230181498224735
128800000 2.260066107756555
128810000 2.259127983306534
128820000 1.9988857222388652
128830000 2.076258494556407
128840000 2.1245531511381404
128850000 2.0860897303436796
128860000 2.007220461843411
128870000 2.1623178448858194
128880000 2.1168001176509126
128890000 2.2092620807080863
128900000 2.1338901405575013
128910000 2.093474108449029
128920000 2.2380512476183174
128930000 2.2940485663433803
128940000 2.3220398198693855
128950000 2.1733719009344976
128960000 2.0228471851452854
128970000 2.1009049716316657
128980000 2.0152946114230157
128990000 1.972119171278775
129000000 2.0411463126083746
129010000 2.0142218203462763
129020000 2.0657288085852397
129030000 1.8705585574425263
129040000 1.9297310125752491
129050000 1.8676034358013538
129060000 1.9304054115415017
129070000 2.1118017032788

131530000 0.046585390499055385
131540000 0.049917569337295166
131550000 0.042716308947980404
131560000 0.04844832537142436
131570000 0.047612553698983456
131580000 0.03890354810770352
131590000 0.041815139932036394
131600000 0.04488989876396127
131610000 0.03643838558594385
131620000 0.044710109813431896
131630000 0.04049918494247065
131640000 0.03870237489407592
131650000 0.04239296707707643
131660000 0.03455232981479168
131670000 0.03482991086742613
131680000 0.037156364210665226
131690000 0.04224420406476657
131700000 0.040449257312582594
131710000 0.03869255613416433
131720000 0.03702729212839736
131730000 0.03693710959449079
131740000 0.036894652737100915
131750000 0.03477871105283499
131760000 0.03348017421329684
131770000 0.03635941978830099
131780000 0.02861360221193896
131790000 0.031406044507410795
131800000 0.032091065213541194
131810000 0.029382998496088716
131820000 0.03194222784494691
131830000 0.029332841299831863
131840000 0.03030378463530541
131850000 0.031267625917838